Este cuaderno es una PoC para comprobar si se puede clasificar codigo vulnerable.


## Datasets:

There  are  two  widely  used  sources  ofvulnerability  data  maintained  by  the  NIST:  the  NVD  which  contains  vulnerabilities  in  production  software,  andthe  SARD  project which  contains  production,  synthetic,and  academic  security  flaws  or  vulnerabilities.  In  the  NVD,each  vulnerability  has  a  unique  Common  Vulnerabilities  andExposures  IDentifier  (CVE  ID)  and  a  Common  WeaknessEnumeration  IDentifier  (CWE  ID)  that  indicates  the  type  ofthe  vulnerability  in  question.  We  collect  the  programs  thatcontain  one  or  multiple  vulnerabilities.  In  the  SARD,  eachprogram (i.e., test case) corresponds to one or multiple CWEIDs because a program can have multiple types of vulnerabil-ities. Therefore, programs with one or multiple CWE IDs arecollected.

https://nvd.nist.gov/vuln/search

https://samate.nist.gov/SARD/search.php 


## Algorihms

Machine  learning  algorithms  cannot  learn  from  raw  text;so  several  steps  of  processing  the  dataset  are  needed  beforetraining  the  algorithm.  First,  the  code  snippets  need  to  beconverted  into  numerical  feature  vectors.  We  used  bag-of-word  model,  a  method  to  represent  each  unique  word  as  afeature. Second, we selected features extracted a subset fromthe dataset and used them for training the machine algorithm.


## Papers y articulos basados:

https://arxiv.org/pdf/1801.01681.pdf

https://arxiv.org/pdf/1803.04497.pdf

https://realpython.com/python-keras-text-classification/

https://arxiv.org/pdf/1809.07945.pdf

https://medium.com/jatana/report-on-text-classification-using-cnn-rnn-han-f0e887214d5f

In [ ]:
import os

# Download dataset and move all .java to one folder
os.system(" find . -type f \( -iname "*.java" \) -exec mv {} all/ \; ")

# Remove comentaries
os.system(" find . \( -type d -name .git -prune \) -o -type f -print0 | xargs -0 sed -i 's/[\/*]//g' ")

# Remove imports
os.system(" find . \( -type d -name .git -prune \) -o -type f -print0 | xargs -0 sed -i 's/^import//g' ")

In [51]:
import pandas as pd
import os

dirlist_sqli_java = os.listdir("datasets/sqli_java/")
print("[*] " + str(len(dirlist)) + " files are importing...")


df_list = []
for file in dirlist_sqli_java:
    df = pd.read_csv("datasets/sqli_java/"+file, names=['code'])
    df['vuln_label'] = 1  # Add another column filled with 1 (vulnerable code)
    df['threat'] = 'sqli_java'
    df_list.append(df)
    
for file in dirlist_sqli_java:
    df = pd.read_csv("datasets/sqli_java/"+file, names=['code'])
    df['vuln_label'] = 0  # Add another column filled with 1 (vulnerable code)
    df['threat'] = 'sqli_java'
    df_list.append(df)
df = pd.concat(df_list)



print(df.iloc[0])
df.head(10)

[*] 3733 files are importing...
code            Licensed to the Apache Software Foundation (...
vuln_label                                                    1
threat                                                sqli_java
Name: 0, dtype: object


,code,vuln_label,threat
0,Licensed to the Apache Software Foundation (...,1,sqli_java
1,or more contributor license agreements. See...,1,sqli_java
2,distributed with this work for additional in...,1,sqli_java
3,regarding copyright ownership. The ASF lice...,1,sqli_java
4,to you under the Apache License,1,sqli_java
5,"""License""); you may not use this file except...",1,sqli_java
6,with the License. You may obtain a copy of ...,1,sqli_java
7,http:www.apache.orglicensesLICENSE-2.0,1,sqli_java
8,Unless required by applicable law or agreed ...,1,sqli_java
9,distributed under the License is distributed...,1,sqli_java


In [52]:
from sklearn.model_selection import train_test_split

df2 = df

# Slipt data

code = df2['code'].values.astype('str')
y = df2['vuln_label'].values

code_train, code_test, y_train, y_test = train_test_split(
            code, y, test_size=0.25, random_state=1000)


# Vectorize
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit(code_train)

X_train = vectorizer.transform(code_train)
X_test  = vectorizer.transform(code_test)
X_train


<1324797x13326 sparse matrix of type '<class 'numpy.int64'>'
	with 3113965 stored elements in Compressed Sparse Row format>

In [53]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train, y_train)
score = model.score(X_test, y_test)

print("Accuracy:", score)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuracy: 0.48420399502716266
